<a href="https://colab.research.google.com/github/Youjin14/data310/blob/master/Project_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [85]:
import tensorflow as tf

import numpy as np
import os
import time

In [86]:
path_to_file = tf.keras.utils.get_file('Bee_Movie1.txt', 'https://github.com/Youjin14/DATA-310-Applied-Machine-Learning/files/5013150/Bee_Movie1.txt')

In [87]:
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print ('Length of text: {} characters'.format(len(text)))

Length of text: 32008 characters


In [84]:
# Take a look at the first 250 characters in text
print(text[:250])

<!DOCTYPE html><html lang="en-US" class="" data-primer><head><link href="https://a.slack-edge.com/f1cf3/style/libs/lato-2-compressed.css" rel="stylesheet" type="text/css" onload="window._cdn ? _cdn.ok(this, arguments) : null" onerror="window._cdn ? _


In [52]:
# The unique characters in the file
vocab = sorted(set(text))
print ('{} unique characters'.format(len(vocab)))

95 unique characters


In [53]:
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

In [54]:
print('{')
for char,_ in zip(char2idx, range(20)):
    print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))
print('  ...\n}')

{
  '\t':   0,
  '\n':   1,
  ' ' :   2,
  '!' :   3,
  '"' :   4,
  '#' :   5,
  '$' :   6,
  '%' :   7,
  '&' :   8,
  "'" :   9,
  '(' :  10,
  ')' :  11,
  '*' :  12,
  '+' :  13,
  ',' :  14,
  '-' :  15,
  '.' :  16,
  '/' :  17,
  '0' :  18,
  '1' :  19,
  ...
}


In [55]:
# Show how the first 13 characters from the text are mapped to integers
print ('{} ---- characters mapped to int ---- > {}'.format(repr(text[:13]), text_as_int[:13]))

'<!DOCTYPE htm' ---- characters mapped to int ---- > [30  3 38 49 37 54 59 50 39  2 73 85 78]


In [56]:
# The maximum length sentence we want for a single input in characters
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
  print(idx2char[i.numpy()])

<
!
D
O
C


In [57]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
  print(repr(''.join(idx2char[item.numpy()])))

'<!DOCTYPE html><html lang="en-US" class="" data-primer><head><link href="https://a.slack-edge.com/f1c'
'f3/style/libs/lato-2-compressed.css" rel="stylesheet" type="text/css" onload="window._cdn ? _cdn.ok(t'
'his, arguments) : null" onerror="window._cdn ? _cdn.failed(this, arguments) : null" crossorigin="anon'
'ymous"><link href="https://a.slack-edge.com/91f11/style/_generic.typography.larsseit.css" rel="styles'
'heet" type="text/css" onload="window._cdn ? _cdn.ok(this, arguments) : null" onerror="window._cdn ? _'


In [58]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [59]:
for input_example, target_example in  dataset.take(1):
  print ('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
  print ('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  '<!DOCTYPE html><html lang="en-US" class="" data-primer><head><link href="https://a.slack-edge.com/f1'
Target data: '!DOCTYPE html><html lang="en-US" class="" data-primer><head><link href="https://a.slack-edge.com/f1c'


In [60]:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    print("Step {:4d}".format(i))
    print("  input: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
    print("  expected output: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))

Step    0
  input: 30 ('<')
  expected output: 3 ('!')
Step    1
  input: 3 ('!')
  expected output: 38 ('D')
Step    2
  input: 38 ('D')
  expected output: 49 ('O')
Step    3
  input: 49 ('O')
  expected output: 37 ('C')
Step    4
  input: 37 ('C')
  expected output: 54 ('T')


In [61]:
# Batch size
BATCH_SIZE = 32

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset shapes: ((32, 100), (32, 100)), types: (tf.int64, tf.int64)>

In [62]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [63]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.GRU(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

In [64]:
model = build_model(
  vocab_size = len(vocab),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

In [65]:
for input_example_batch, target_example_batch in dataset.take(1):
  example_batch_predictions = model(input_example_batch)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(32, 100, 95) # (batch_size, sequence_length, vocab_size)


In [66]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (32, None, 256)           24320     
_________________________________________________________________
gru_1 (GRU)                  (32, None, 1024)          3938304   
_________________________________________________________________
dense_1 (Dense)              (32, None, 95)            97375     
Total params: 4,059,999
Trainable params: 4,059,999
Non-trainable params: 0
_________________________________________________________________


In [67]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [68]:
sampled_indices

array([ 5, 85, 68, 37, 55, 32, 47, 92, 30, 78, 77, 25, 30, 87, 32, 26, 50,
       66, 57, 53, 57, 10, 87, 13, 67, 51,  4, 69,  7, 41, 12, 38, 33, 55,
       15, 67, 24, 75, 57, 81, 43, 51, 40, 18,  2, 18,  0, 48, 33, 68, 76,
       58, 91, 78, 22, 72, 25, 80, 68, 17, 35, 71, 85,  3, 79, 14, 54, 23,
       76,  1, 87, 53, 45, 75, 65, 85, 44,  1,  6, 45, 24, 70, 34, 51, 48,
       13, 29, 24, 83,  5, 78, 39, 60, 90, 17, 77, 28, 24, 46, 34])

In [69]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss  = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (32, 100, 95)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.5541244


In [70]:
model.compile(optimizer='adam', loss=loss)

In [71]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [72]:
EPOCHS=100

In [73]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/100
9/9 [==============================] - 0s 51ms/step - loss: 4.3299
Epoch 2/100
9/9 [==============================] - 0s 38ms/step - loss: 3.8525
Epoch 3/100
9/9 [==============================] - 0s 38ms/step - loss: 3.4976
Epoch 4/100
9/9 [==============================] - 0s 39ms/step - loss: 3.2716
Epoch 5/100
9/9 [==============================] - 0s 38ms/step - loss: 3.0545
Epoch 6/100
9/9 [==============================] - 0s 44ms/step - loss: 2.8347
Epoch 7/100
9/9 [==============================] - 0s 39ms/step - loss: 2.6580
Epoch 8/100
9/9 [==============================] - 0s 38ms/step - loss: 2.5173
Epoch 9/100
9/9 [==============================] - 0s 40ms/step - loss: 2.3993
Epoch 10/100
9/9 [==============================] - 0s 40ms/step - loss: 2.3129
Epoch 11/100
9/9 [==============================] - 0s 45ms/step - loss: 2.2122
Epoch 12/100
9/9 [==============================] - 0s 39ms/step - loss: 2.1505
Epoch 13/100
9/9 [==============================]

In [74]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints/ckpt_100'

In [75]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

In [76]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (1, None, 256)            24320     
_________________________________________________________________
gru_2 (GRU)                  (1, None, 1024)           3938304   
_________________________________________________________________
dense_2 (Dense)              (1, None, 95)             97375     
Total params: 4,059,999
Trainable params: 4,059,999
Non-trainable params: 0
_________________________________________________________________


In [77]:
def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 1000

  # Converting our start string to numbers (vectorizing)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 1.0

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # remove the batch dimension
      predictions = tf.squeeze(predictions, 0)

      # using a categorical distribution to predict the character returned by the model
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # We pass the predicted character as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [78]:
print(generate_text(model, start_string=u"Barry: "))

Barry: revale_page_get":false,"feature_shared_channels_multi_org_mpim":true,"feature_select_on_tab":true,"feature_desktop_relse if (node.nodeName === 'IMG') {
				nodeSrc = node.src || node.currentSrc;

				gtmea6e;
	mar var endpoint_url = window.ts_endpoint_url || 'https://slack.com/beacon/timing';
			new Image().src = endpoint_url + '?data=' + encoment] = timGom && date.getT${eor (!SOme {megser_dication":true,"feature_shared_channels_multi_email_fiixperiment_assiggments":{},"noz":false,"feature_grobals_update":true,"feature_sk_data_table_a11y":false,"feature_sk_loading_button_motions":true,"feature_sk_date_apdays_api":true,"feature_shared_channels_invite_livet_opt":false,"feature_builder_message_step_rich_text":true,"feature_select_on_tab":true,"feature_org_level_apps":false,"feature_apps_manage_permissions_scope_changes":true,"feature_rooms_join_api":false,"feature_locale_ru_RU":false,"feature_plus_menu_add_apps_link":false,"feature_shared_channels_emoji_delight":false,"feature_set_